In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries/ Modules

In [ ]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3 as sql
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 2000)
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Read the CSV files using Pandas

In [ ]:
transaction = pd.read_csv(r"../input/forage-customer-data-and-segments/QVI_transaction_data.csv")
behaviour = pd.read_csv("../input/purchase-behavior-dataset/purchase behaviou dataset.csv")

In [ ]:
x = pd.DataFrame(behaviour)
print(x)

# See Columns in trasaction and Behavior Table

In [ ]:
for x in transaction:
    print (x)

In [ ]:
for y in behaviour:
    print(y)

# Explore the data of each table

In [ ]:
print("Below shows the datatypes and null counts in tables")
print(transaction.info())
print(behaviour.info())

print("Below shows the count of unique values in tables")
print(transaction.nunique())
print(behaviour.nunique())

# Create a database and store the csv data in the database. 
# Here the database will be named "forage"

In [ ]:
forage='forage.db'
connection=sql.connect(forage)

# Create cursor object to execute queries in the database.

In [ ]:
cursor=connection.cursor()

# **Make table in the database and input the csv file data into the tables.**

In [ ]:
cursor.execute("""CREATE TABLE tran
(DATE int,
STORE_NBR int,
LYLTY_CARD_NBR int,
TXN_ID int,
PROD_NBR int,
PROD_NAME text,
PROD_QTY int,
TOT_SALES int)""")

In [ ]:
cursor.execute("""
CREATE TABLE behave
(LYLTY_CARD_NBR integer,
LIFESTAGE object,
PREMIUM_CUSTOMER object)""")

# Transfer data from csv files to the SQL database forage

In [ ]:
transaction.to_sql("tran",connection,if_exists='append',index=False)
behaviour.to_sql("behave", connection, if_exists='append', index=False)

# **Join the two tables on common column which is LYLTY_CARD_NBR**

In [ ]:
cursor.execute("""CREATE TABLE customer_behaviour AS
SELECT *
FROM tran
JOIN behave ON tran.LYLTY_CARD_NBR = behave.LYLTY_CARD_NBR""")

In [ ]:
df=pd.read_sql_query("""SELECT * FROM customer_behaviour""", connection)
print(df.head())

# Import pandas profiling module for easy EDA

In [ ]:
import pandas_profiling
%pip install ipywidgets

In [ ]:
data = pd.read_sql_query("""SELECT * FROM customer_behaviour""", connection)

In [ ]:
data.profile_report()

# Highest sold product

In [ ]:
top_sold = pd.read_sql_query("""
SELECT PROD_NBR,PROD_NAME,SUM(TOT_SALES) AS SALES
FROM customer_behaviour 
GROUP BY PROD_NBR
ORDER BY SALES DESC
LIMIT 10""",connection)

print(top_sold)

#sales_by_qty = sales_by_qty.sort_values(by="SALES") --- To arrange the graph by adjusting the columns
vis=px.bar(top_sold, x=top_sold.PROD_NAME, y=top_sold.SALES, color = top_sold.SALES)
vis.show()

# HIGHEST SOLD PRODUCT BY QUANTITY

In [ ]:
sales_by_qty=pd.read_sql_query("""
SELECT PROD_QTY,PROD_NBR, PROD_NAME, SUM(TOT_SALES) AS SALES
FROM customer_behaviour
GROUP BY PROD_QTY, PROD_NBR
ORDER BY SALES DESC
LIMIT 100""",connection)

print(sales_by_qty)

In [ ]:
qty_breakdown =px.line(sales_by_qty, x=sales_by_qty.PROD_NAME, y=sales_by_qty.SALES, color=sales_by_qty.PROD_QTY)
qty_breakdown.show()
print ("Highest sold products are sold in pair (QTY-2)")


# SALES BY CUSTOMER SEGMENT

In [ ]:
customer_contribution = pd.read_sql_query("""
SELECT LIFESTAGE, PROD_NAME, SUM(TOT_SALES) AS SALES
FROM customer_behaviour
GROUP BY LIFESTAGE, PROD_NAME
ORDER BY SALES DESC
LIMIT 10""", connection)

print(customer_contribution)

In [ ]:
customer_segment = px.bar(customer_contribution, x=customer_contribution.LIFESTAGE,y= customer_contribution.SALES, color=customer_contribution.PROD_NAME)
customer_segment.show()
print("Older singles/ couples contribute highest towards top 10 selling products")

In [ ]:
sales_contribution_by_segment = pd.read_sql_query("""
SELECT LIFESTAGE, SUM(TOT_SALES) AS SALES
FROM customer_behaviour
GROUP BY LIFESTAGE
ORDER BY SALES DESC""", connection)

print(sales_contribution_by_segment)

In [ ]:
sales_per_segment = px.pie(sales_contribution_by_segment, values=sales_contribution_by_segment.SALES, names = sales_contribution_by_segment.LIFESTAGE)
sales_per_segment.show()

# Conclusion

The correlations in the explored data gave insights to relations between sales of different products and segments that are contributing to the sales. 

1. Highest Sold Product
List of top 10 products in terms of sales were visualized. Product number 4 was the top selling product that made the highest sale of approximately 40.3K. Product 32 was the top 10th sold product with the sale of 32.5K. 


2. Product sale by Quantity
A trend of products sold in a pair of 2 were seen to make highest contribution to sales. The top 100 sold products were found with highest sales were sold in a **pair of 2** according to the trend visualized. 

3. Sales by customer segment
The top sold product Dorito corn chip was bought mostly by OLDER SINGLES/ COUPLES. In addition, OLDER SINGLES/ COUPLES have contributed significanty to oveall sales of different products in the markets. 

4. Customer contributions to sales. 
It is observed that older singles/ couples segment of customers have contributed almost 21% to total sales according to the data followed by retirees customer data. 
